### 数据获取

#### 网络数据获取 Request包

In [45]:
import requests
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/78.0.3904.108 Safari/537.36'}
r = requests.get("https://book.douban.com/subject/1084336/comments/hot?p=1",headers=headers)
# r = requests.get("https://www.baidu.com")
r.status_code

200

In [28]:
print(r.encoding)

utf-8


In [46]:
 r.text

'\n\n<!DOCTYPE html>\n<html lang="zh-cmn-Hans" class="ua-windows ua-webkit book-new-nav">\n<head>\n  <meta http-equiv="Content-Type" content="text/html; charset=utf-8">\n  <title>\n    小王子 短评\n</title>\n  \n<script>!function(e){var o=function(o,n,t){var c,i,r=new Date;n=n||30,t=t||"/",r.setTime(r.getTime()+24*n*60*60*1e3),c="; expires="+r.toGMTString();for(i in o)e.cookie=i+"="+o[i]+c+"; path="+t},n=function(o){var n,t,c,i=o+"=",r=e.cookie.split(";");for(t=0,c=r.length;t<c;t++)if(n=r[t].replace(/^\\s+|\\s+$/g,""),0==n.indexOf(i))return n.substring(i.length,n.length).replace(/\\"/g,"");return null},t=e.write,c={"douban.com":1,"douban.fm":1,"google.com":1,"google.cn":1,"googleapis.com":1,"gmaptiles.co.kr":1,"gstatic.com":1,"gstatic.cn":1,"google-analytics.com":1,"googleadservices.com":1},i=function(e,o){var n=new Image;n.onload=function(){},n.src="https://www.douban.com/j/except_report?kind=ra022&reason="+encodeURIComponent(e)+"&environment="+encodeURIComponent(o)},r=function(o){try{t.ca

#### 正则表达式

In [31]:
import re;
# help(re)

In [33]:
#用 \s 匹配空白符，换行符也属于空白符，空白符并不是空格
pattern = re.compile('\stest')
p = re.findall(pattern, 'span is \ntest')
p

#### 正则表达式字符   () 的用法

In [43]:
#)() 可以捕获符合的是否有固定的文本，此处捕获字符开始为 he的内容
pattern = re.compile('^(he)+')
p =re.findall(pattern, 'heheh he heh')
p

['he']

In [44]:
#)() 可以捕获符合的非固定内容，此处捕获allstar 后边的内容
testtext = '<span class="user-stars allstar45 rating"'
pattern = re.compile('<span class="user-stars allstar(.*?) rating"')
p = re.findall(pattern, testtext)
p

['45']

## BeautifulSoup

### 重要对象 Tag , NavigableString , BeautifulSoup , Comment

#### tag

In [25]:
soup = BeautifulSoup('<span><b class="boldest">Extremely bold</b></span>')
tag = soup.span.b
print(tag)
print(tag.name)
type(tag)
# <class 'bs4.element.Tag'>

<b class="boldest">Extremely bold</b>
b


bs4.element.Tag

##### attrs 

In [30]:
soup = BeautifulSoup('<blockquote class="verybold" id="1">Extremely bold</blockquote>', 'lxml')
soup.blockquote.attrs

{'class': ['verybold'], 'id': '1'}

##### 多值属性 最常见的多值的属性是 class 

In [29]:
css_soup = BeautifulSoup('<p class="body strikeout"></p>')
print(css_soup.p.attrs['class'])
css_soup.p.attrs['class'].append('midlight')
print(css_soup.p.attrs['class'])

['body', 'strikeout']
['body', 'strikeout', 'midlight']


#### ResultSet

In [22]:
from bs4 import BeautifulSoup
html_doc = """
<html><head><title>The Dormouse's story</title></head>
<body>
<p class="title"><b>The Dormouse's story</b></p>

<p class="story">Once upon a time there were three little sisters; and their names were
<a href="http://example.com/elsie" class="sister" id="link1">Elsie</a>,
<a href="http://example.com/lacie" class="sister" id="link2">Lacie</a> and
<a href="http://example.com/tillie" class="sister" id="link3">Tillie</a>;
and they lived at the bottom of a well.</p>

<p class="story">...</p>
NavigableString:I'm here,Mr C
"""
soup = BeautifulSoup(html_doc, 'lxml')
story = soup.find_all('')
story = soup.find_all('p', 'story')
# story = soup.find_all('title')

In [23]:
print(soup.html.head)#soup.tagname的方式直接按层级方式访问Tag
print('-'*100)
print(type(story))
for item in story:
#     print(type(item))
    print(item)
    print('-'*100)

<head><title>The Dormouse's story</title></head>
----------------------------------------------------------------------------------------------------
<class 'bs4.element.ResultSet'>


In [ ]:
# 豆瓣短评前50的短评和评分均值

In [61]:
import requests, re, time
from bs4 import BeautifulSoup

count = 0
i = 0
s, count_s, count_del = 0, 0, 0
lst_stars = []
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/78.0.3904.108 Safari/537.36'}
while count < 50:
    try:
        r = requests.get('https://book.douban.com/subject/1084336/comments/hot?p=' + str(i+1), headers = headers)
    except Exception as err:
        print(err)
        break
    soup = BeautifulSoup(r.text, 'lxml')
    comments = soup.find_all('span', 'short')
    pattern = re.compile('<span class="user-stars allstar(.*?) rating"')
    p = re.findall(pattern, r.text)
    for item in comments:
        count += 1
        if count > 50:
            count_del += 1     # count the number of comments more than 50 of the page 
        else:
            print(count, item.string)
    for star in p:
        lst_stars.append(int(star))
    time.sleep(5)    # delay request from douban's robots.txt
    i += 1
    for star in lst_stars[:-count_del]:   # calculate the rating star of 50 comments
        s += int(star) 
if count >= 50:
    print(s // (len(lst_stars)-count_del))

1 十几岁的时候渴慕着小王子，一天之间可以看四十四次日落。是在多久之后才明白，看四十四次日落的小王子，他有多么难过。
2 读了好多年，终于读完了，但是实在共鸣不起来，虽然知道那些道理，但真的觉得没什么了不起啊，是我还太幼稚吗？
3 我早该猜到，在她那可笑的伎俩后面是缱绻柔情啊。花朵是如此的天真无邪，可是，我毕竟太年轻了，不知该如何去爱她。
4 我的玫瑰花儿，只有四个微不足道的刺，用来抵御这个世界。
5 这是一本定位颇尴尬的名著，薄薄的一本小册子里充满了太多的暗喻和说教，对孩子来说看不懂且沉闷，对大人来说，也许能够理解其中一部分的暗喻了，但是却又早已成为了书中的“成年人”。
6 说实话 我看不太懂 但还是跟风给个5星吧 以显示我也是有思想有学识之人
7 虽然我实在幼稚，但我并不怎么喜欢孩童般的纯净，我只爱风浪过后的平静，流水打磨出的清亮，大雪纷飞时的安宁，沧桑看透的纯真，我爱眼冷心热的庄子，人究竟无所逃于天地之间，各适其性，做自己认为有意义的事就好了，无论是统治宇宙还是喝酒点灯，对他们来说，都比爱一朵玫瑰重要，这不是很好的么
8 不能理解的是，為什么它忽然紅成這樣？
9 他像一颗树那样倒了下去
10 狐狸告诉小王子的秘密是：用心去看才看得清楚；是分离让小王子更思念他的玫瑰；爱就是责任。 

11 我老觉得小狐狸跟小王子是在搞GAY
12 一边读一边很想哭。太寂寞了。人类真的太寂寞了。
13 It is the time you have wasted for your rose that makes your rose so important.
14 原来在我还不懂爱情的时候就爱上了你
15 第一遍读时，我才4岁。等到真的读懂，才明白为什么这是一部“童话”。
16 如果你是小王子，我就是那捉鸡的狐狸，求你驯养了我，却又对你说要你回去对你的玫瑰花负责。还大方地以为只要看见了麦田就会想起你头发的颜色。我却永远无法随你回到B-612星球。现在觉得，我可能只是猴面包树而已。
17 痛苦迷茫不是因为成为了“可笑”的大人，而是成为大人却没有真正长大。所以回过头来想要从怀念童年中解脱缓解痛苦那是本末倒置的做法。如果你作为一个成年人觉得痛苦，原因不是因为你“成年”了，也不是因为是生而为“人”，而是“你”停止了思考停止了学习。
18 是你浪费在我身上的时间 使我变得 如此珍贵
19 